In [5]:
from typing import List, Optional, Tuple

from linked_list_helper import ListNode, makeListOfLinkedLists

287. Find the Duplicate Number


In [2]:
class Solution:
    def findDuplicate(self, nums: List[int]) -> int:
        # linked list cycle if you use the values as pointers to an index.
        # where slow and fast meet will be at a distance from cycle start
        #   equal to distance of first index from cycle start
        # k = m + p - c
        # 2k = m + 2p - c
        # k is steps taken
        # m is length until cycle start
        # p is length of cycle
        # c is steps from intersection of slow and fast to the cycle start (if at cycle start then you are p-steps away)
        #
        # first identity is taken from `slow`
        # second is taken from `fast`

        slow = fast = 0  # zero index (pointers)
        # cycle will never start from index 0 because nums[i] != 0
        while nums[slow] != nums[fast] or slow == 0:
            slow = nums[slow]  # slow.next essentially
            fast = nums[nums[fast]]  # fast.next.next essentially
        # now slow and fast are at distance 'c' = 'm' from start of cycle which is where the duplicate number is
        start = 0  # at distance 'm' = 'c' from the start of cycle

        while nums[slow] != nums[start]:
            start = nums[start]
            slow = nums[slow]

        return nums[slow]


findDuplicate = Solution()
print(findDuplicate.findDuplicate([1, 3, 4, 2, 2]))
print(findDuplicate.findDuplicate([3, 1, 3, 4, 2]))

2
3


**138. Copy List with Random Pointer**

deep copy


In [3]:
class Node:
    def __init__(self, x: int, next: "Node" = None, random: "Node" = None):
        self.val = int(x)
        self.next = next
        self.random = random

    def __repr__(self) -> str:
        return str(self.val)


class Solution:  # 94% time, 78% memory
    def copyRandomList(self, head: Optional[Node]) -> Optional[Node]:
        # the None: None is necessary because this won't get populated automatically
        dct = {None: None}  # dict[Node, Node] (original node -> copy node)
        # first pass populate the dict:
        cur = head

        while cur:
            dct[cur] = Node(cur.val)
            cur = cur.next

        newHead = Node(0)
        dummy = newHead
        # second pass populate the new list with and next and random
        for key, val in dct.items():
            if key:
                dummy.next = val
                dummy.next.next = dct[key.next]
                dummy.next.random = dct[key.random]
                dummy = dummy.next
        return newHead.next


node1 = Node(1)
node2 = Node(2)
node1.next = node2
node1.random = node2
node2.random = node2

copyRandomList = Solution()
root = copyRandomList.copyRandomList(node1)

**146. LRU Cache**


In [4]:
# 84% time, 16% memory
# O(1) time and O(1) memory for the get and put operations

# Your LRUCache object will be instantiated and called as such:
# obj = LRUCache(capacity)
# param_1 = obj.get(key)
# obj.put(key,value)
class LRUNode:
    def __init__(self, key: int, value: int, next=None, prev=None):
        "Doubly linked list node"
        self.key = key  # for knowing which key to evict
        self.value = value
        self.next = next
        self.prev = prev

    def __repr__(self) -> str:
        return f"LRUNode[{self.key}, {self.value}]"

    def __str__(self) -> str:
        return f"LRUNode: k={self.key}, v={self.value}"


class LRUCache:
    def __init__(self, capacity: int) -> None:
        self._capacity = capacity
        self.cache = {}  # dict where the values of the keys are pointers to the nodes generated
        # the nodes are in a double linked list
        self._least_recent: LRUNode = LRUNode(
            "LRU", "LRU"
        )  # used as a first head node that points to least recent
        self._most_recent: LRUNode = LRUNode(
            "MRU", "MRU"
        )  # used as a second head node that points to most recent

        self._least_recent.next = self._most_recent
        self._most_recent.prev = self._least_recent

    def display_linked_list(self) -> str:
        cur_node = self._least_recent
        print("\nForward")
        while cur_node:
            print(cur_node, end=" -> ")
            cur_node = cur_node.next
        print("\nBackward")
        cur_node = self._most_recent
        while cur_node:
            print(cur_node, end=" -> ")
            cur_node = cur_node.prev
        print("\n")

    def _remove_node_and_rearrange_adjacents(self, node: LRUNode) -> int:
        "reconnects `node`'s adjacent nodes and returns the `key` attribute of `node`"
        # update the node before and after fetched node
        # this should always work even if the fetched node's `prev` is `self._least_recent` or `next` is most_recent
        node.prev.next = node.next
        node.next.prev = node.prev
        return node.key

    def _insert_node_at_most_recent_slot(self, node: LRUNode):
        # update the fetched node's next and prev pointers:
        node.next = self._most_recent
        node.prev = self._most_recent.prev
        # update _most_recent by making the previous most recently used node's `next` point to the fetched node
        # and by making _most_recent.prev point to the fetched node
        self._most_recent.prev.next = node
        self._most_recent.prev = node

    def get(self, key: int) -> int:
        if key in self.cache:
            self._remove_node_and_rearrange_adjacents(self.cache[key])
            self._insert_node_at_most_recent_slot(self.cache[key])
            return self.cache[key].value
        return -1

    def put(self, key: int, value: int) -> None:
        if key in self.cache:
            self._remove_node_and_rearrange_adjacents(self.cache[key])
        self.cache[key] = LRUNode(key, value)
        self._insert_node_at_most_recent_slot(self.cache[key])

        if len(self.cache) > self._capacity:
            remove_key = self._remove_node_and_rearrange_adjacents(
                self._least_recent.next
            )
            del self.cache[remove_key]

In [5]:
lru_cache = LRUCache(2)

# adding one node
lru_cache.put(1, 1)
print("cache[1]:", lru_cache.get(1))
lru_cache.display_linked_list()

# adding a second node
lru_cache.put(2, 2)
print("cache[2]:", lru_cache.get(2))
lru_cache.display_linked_list()

# modifying MRU and LRU
print("cache[1]:", lru_cache.get(1))
lru_cache.display_linked_list()

# evicting LRU by adding third node
lru_cache.put(4, 4)
print("cache[4]:", lru_cache.get(4))
print("cache[2]:", lru_cache.get(2))  # should be evicted
lru_cache.display_linked_list()

lru_cache.put(4, 1)
print("cache[4]:", lru_cache.get(4))
lru_cache.display_linked_list()

cache[1]: 1

Forward
LRUNode: k=LRU, v=LRU -> LRUNode: k=1, v=1 -> LRUNode: k=MRU, v=MRU -> 
Backward
LRUNode: k=MRU, v=MRU -> LRUNode: k=1, v=1 -> LRUNode: k=LRU, v=LRU -> 

cache[2]: 2

Forward
LRUNode: k=LRU, v=LRU -> LRUNode: k=1, v=1 -> LRUNode: k=2, v=2 -> LRUNode: k=MRU, v=MRU -> 
Backward
LRUNode: k=MRU, v=MRU -> LRUNode: k=2, v=2 -> LRUNode: k=1, v=1 -> LRUNode: k=LRU, v=LRU -> 

cache[1]: 1

Forward
LRUNode: k=LRU, v=LRU -> LRUNode: k=2, v=2 -> LRUNode: k=1, v=1 -> LRUNode: k=MRU, v=MRU -> 
Backward
LRUNode: k=MRU, v=MRU -> LRUNode: k=1, v=1 -> LRUNode: k=2, v=2 -> LRUNode: k=LRU, v=LRU -> 

cache[4]: 4
cache[2]: -1

Forward
LRUNode: k=LRU, v=LRU -> LRUNode: k=1, v=1 -> LRUNode: k=4, v=4 -> LRUNode: k=MRU, v=MRU -> 
Backward
LRUNode: k=MRU, v=MRU -> LRUNode: k=4, v=4 -> LRUNode: k=1, v=1 -> LRUNode: k=LRU, v=LRU -> 

cache[4]: 1

Forward
LRUNode: k=LRU, v=LRU -> LRUNode: k=1, v=1 -> LRUNode: k=4, v=1 -> LRUNode: k=MRU, v=MRU -> 
Backward
LRUNode: k=MRU, v=MRU -> LRUNode: k=4,

**641. Design Circular Deque**

62% time, 7% memory


In [6]:
class Node:
    def __init__(self, value, next=None, prev=None):
        self.value = value
        self.next = next
        self.prev = prev


class MyCircularDeque:
    def __init__(self, k: int):
        # Very similar to LRU cache, with a node pointing at head and a node pointing at tail
        self.FRONT = Node(-1)
        self.BACK = Node(-1, next=self.FRONT)
        self.FRONT.prev = self.BACK

        self.k = k
        self.cur_l = 0

    def insertFront(self, value: int) -> bool:
        if self.cur_l == self.k:
            return False
        self.cur_l += 1
        node = Node(value=value, next=self.FRONT, prev=self.FRONT.prev)
        self.FRONT.prev.next = node
        self.FRONT.prev = node
        return True

    def insertLast(self, value: int) -> bool:
        if self.cur_l == self.k:
            return False
        self.cur_l += 1
        node = Node(value=value, next=self.BACK.next, prev=self.BACK)
        self.BACK.next.prev = node
        self.BACK.next = node
        return True

    def deleteFront(self) -> bool:
        if not self.cur_l:
            return False
        self.cur_l -= 1
        self.FRONT.prev = self.FRONT.prev.prev
        self.FRONT.prev.next = self.FRONT
        return True

    def deleteLast(self) -> bool:
        if not self.cur_l:
            return False
        self.cur_l -= 1
        self.BACK.next = self.BACK.next.next
        self.BACK.next.prev = self.BACK
        return True

    def getFront(self) -> int:
        return self.FRONT.prev.value

    def getRear(self) -> int:
        return self.BACK.next.value

    def isEmpty(self) -> bool:
        return self.cur_l == 0

    def isFull(self) -> bool:
        return self.cur_l == self.k

    def __str__(self):
        cur = self.FRONT.prev
        s = ["FRONT"]
        while cur.prev:
            s.append(f"{cur.value}")
            cur = cur.prev
        s.append("BACK")
        return " <- ".join(s)


# Your MyCircularDeque object will be instantiated and called as such:
obj = MyCircularDeque(6)
param_1 = obj.insertFront(8)
param_2 = obj.insertLast(7)
param_3 = obj.insertLast(6)
param_4 = obj.insertLast(100)
obj.deleteLast()
param_5 = obj.insertFront(99)
param_6 = obj.insertFront(9)
param_7 = obj.insertFront(5)
param_8 = obj.insertFront(10)
print(param_8)
# param_3 = obj.deleteNext()
# param_4 = obj.deleteLast()
param_9 = obj.getFront()
print(param_9)

# param_6 = obj.getRear()
# param_7 = obj.isEmpty()
param_8 = obj.isFull()
print(param_8)
print(obj)

False
5
True
FRONT <- 5 <- 9 <- 99 <- 8 <- 7 <- 6 <- BACK


**27. Merge k sorted lists**

91.4% time, 88.7% memory


In [ ]:
import heapq
from time import perf_counter


class Solution:
    def heapPush(self, heap, lists, idx) -> None:
        if lists[idx] is not None:
            heapq.heappush(heap, (lists[idx].val, idx))  # type: ignore
            lists[idx] = lists[idx].next  # type: ignore

    def mergeKLists(self, lists: List[Optional[ListNode]]) -> Optional[ListNode]:
        """
        First instinct is initialize a min heap of size k
        It will be used to dictate when to stop appending from the current linked list
        Heap will need to maintain node's value (for sorting) and the index of the linked list in `lists`
        There will be no comparison collisions because there is no (node value, lists index) pair duplicate
        """
        if len(lists) == 1:
            # edge case of them giving us the answer
            return lists[0]

        heap: List[Tuple[int, int]] = []
        heapq.heapify(heap)
        for i in range(len(lists)):
            if lists[i] is not None:
                heapq.heappush(heap, (lists[i].val, i))  # type: ignore
                lists[i] = lists[i].next  # type: ignore

        if not heap:
            # edge case of no linked lists in lists
            return None

        # init root
        val, idx = heapq.heappop(heap)
        self.heapPush(heap, lists, idx)
        root = cur = ListNode(val)

        while heap:
            # while the current linked list has nodes and is less than the smalles value in the heap:
            #   keep appending and move the current node and update current linked list
            while not heap or (lists[idx] is not None and lists[idx].val <= heap[0][0]):  # type: ignore
                cur.next = ListNode(lists[idx].val)  # type: ignore
                lists[idx] = lists[idx].next  # type: ignore
                cur = cur.next

            # if there are nodes remaining in the current linked list:
            #   push to heap
            self.heapPush(heap, lists, idx)

            # grab next value from heap and then add back to heap (if you can)
            val, idx = heapq.heappop(heap)
            self.heapPush(heap, lists, idx)

            # update cur
            cur.next = ListNode(val)
            cur = cur.next

        return root


class ListNodeWrapper:
    """
    The class that NeetCode used to push nodes to the heap.
    """

    def __init__(self, node: ListNode):
        self.node = node

    def __lt__(self, other: "ListNodeWrapper"):
        return self.node.val < other.node.val


# solve this later:
class SolutionCleanerCode:
    """
    Based on NeetCode's Heap solution implementation
    """

    def mergeKLists(self, lists: List[Optional[ListNode]]) -> Optional[ListNode]:
        """
        First instinct is initialize a min heap of size k
        It will be used to dictate when to stop appending from the current linked list
        Heap will need to maintain node's value (for sorting) and the index of the linked list in `lists`
        There will be no comparison collisions because there is no (node value, lists index) pair duplicate
        """
        if len(lists) == 0:
            return None

        heap: List[Tuple[int, int]] = []
        for i in range(len(lists)):
            if lists[i] is not None:
                heapq.heappush(heap, (lists[i].val, i))  # type: ignore
                lists[i] = lists[i].next  # type: ignore

        # init root
        root = cur = ListNode(0)

        while heap:
            # get next minimum
            val, idx = heapq.heappop(heap)
            cur.next = ListNode(val)
            cur = cur.next

            if lists[idx] is not None:
                heapq.heappush(heap, (lists[idx].val, idx))  # type: ignore
                lists[idx] = lists[idx].next  # type: ignore

        return root.next


mergeKLists = Solution()

mergeKListsCleaner = SolutionCleanerCode()


for lists in (
    [[1, 4, 5], [1, 3, 4], [2, 6]],
    [],
    [[]],
    [[1, 2, 2], [1, 1, 2]],
    [[-2, -1, -1, -1], []],
):
    print(lists)
    start = perf_counter()
    print("\t", mergeKLists.mergeKLists(makeListOfLinkedLists(lists)))
    print("\t", perf_counter() - start)
    start = perf_counter()
    print("\t", mergeKListsCleaner.mergeKLists(makeListOfLinkedLists(lists)))
    print("\t", perf_counter() - start)

[[1, 4, 5], [1, 3, 4], [2, 6]]
	 1 -> 1 -> 2 -> 3 -> 4 -> 4 -> 5 -> 6
	 0.0001167919999716105
	 1 -> 1 -> 2 -> 3 -> 4 -> 4 -> 5 -> 6
	 6.65000000026339e-05
[]
	 None
	 2.712500008783536e-05
	 None
	 2.3625000039828592e-05
[[]]
	 None
	 2.637500006130722e-05
	 None
	 2.7416999955676147e-05
[[1, 2, 2], [1, 1, 2]]
	 1 -> 1 -> 1 -> 2 -> 2 -> 2
	 0.00011050000011891825
	 1 -> 1 -> 1 -> 2 -> 2 -> 2
	 8.237499991992081e-05
[[-2, -1, -1, -1], []]
	 -2 -> -1 -> -1 -> -1
	 5.2583000069716945e-05
	 -2 -> -1 -> -1 -> -1
	 4.279099994164426e-05
